In [223]:
import os 

def build_label_dict():
    d = {}
    
    label_path = r"../Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = int(v.strip())
    return d

Create vectors from features

In [254]:
import json
import numpy as np

# NOT IN USE RIGHT NOW
def compute_sum_ts(prev_ts, cur_ts, sum_ts):
    if cur_ts != 0:
        sum_ts += cur_ts - prev_ts
    return sum_ts


def update_ts(chat, i, cur_ts):
    prev_ts = cur_ts
    cur_ts = chat['turns'][i]['timestamp']
    return prev_ts, cur_ts

def update_sent_len(chat, i, avg_len_speaker, avg_len_wizard):
    if chat['turns'][i]['author'] == 'user':
        avg_len_speaker += len(chat['turns'][i]['text'].split())
    else:
        avg_len_wizard += len(chat['turns'][i]['text'].split())
    return avg_len_speaker, avg_len_wizard

def gen_vector(k, chat):
    tss = []
    is_budget = 0
    avg_len_user, avg_len_wizard = 0, 0 # aveage sent len of user and wizard
    cur_ts = first_ts = chat['turns'][0]['timestamp']
    prev_ts = 0
    for i in range( k ):
        # sent len feature
        avg_len_user, avg_len_wizard = update_sent_len(chat, i, avg_len_user, avg_len_wizard) # update avg sent len
        # timelapse feature
        prev_ts, cur_ts = update_ts(chat, i, cur_ts)
        if i > 0:
            tss.append( cur_ts - prev_ts )# start with the diff from first to second frame
        # budget feature
        for arg in chat['turns'][i]['labels']['acts']:
            for inner_dict in arg['args']:
                if  inner_dict['key'] == 'budget':
                    is_budget = 1 # talking about the budget
    
    # total time feature
    total_time = (prev_ts - first_ts) / pow(10,6) # chat total time normalized
    avg_len_user = avg_len_user * 2 / (100 * k)
    avg_len_wizard = avg_len_wizard * 2 / (100 * k)
    tss = [x / pow(10,6) for x in tss]
    tss.extend([total_time, avg_len_user, avg_len_wizard, is_budget]) # list of all features
    chat_vec = np.array(tss) # gen feature vector
    return chat_vec

def gen_data(d, k):
    chat_path = r"../Frames-dataset/chats"
    chat_features = []
    labels = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        if len(chat['turns']) >= k: # skip really short chats
                labels.append(d[filename])
                chat_features.append( gen_vector(k, chat) )
    return np.array(chat_features), labels

In [291]:
from keras.layers import Input, Dense, merge, Embedding, Activation
from keras.models import Sequential
TRAIN_SIZE = 1000
K = 4 # number of sentences from start of chat
INPUT_DIM = K + 3

# Generate data
d = build_label_dict()
X, Y = gen_data(d, K)

# Divide into training and test data
x_train, y_train = X[:TRAIN_SIZE], Y[:TRAIN_SIZE]
x_test, y_test = X[TRAIN_SIZE:], Y[TRAIN_SIZE:]

# Define the model
model = Sequential()
model.add(Dense(12, input_dim=INPUT_DIM, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

# Compile and fit
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=0)

# Test the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_334 (Dense)            (None, 12)                96        
_________________________________________________________________
dense_335 (Dense)            (None, 8)                 104       
_________________________________________________________________
dense_336 (Dense)            (None, 1)                 9         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________
None

acc: 64.63%
